In [1]:
# to save output in log file #
# ---------------------------- #
##############################################
import sys
import logging

nblog = open("train1.log", "w+")

sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 50
##############################################


# ---------------- #
# Imported Modules #
# ---------------- #
import os
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")
#sys.path.insert(1, '/home/maxim/Projects/pyseqm_d/My_d_combined/PYSEQM_dev/')


### path to HIPNN ###
sys.path.append('/home/maxim/Projects/hipnn/hippynn_restricted/')

import numpy as np
import torch
from hippynn.interfaces.pyseqm_interface.seqm_nodes import *
from hippynn.interfaces.pyseqm_interface.callback import update_scf_eps, save_and_stop_after
import hippynn.interfaces.pyseqm_interface
import hippynn
from hippynn.graphs import inputs, networks, targets, physics
from hippynn.graphs import loss
from hippynn import plotting
from hippynn.databases import DirectoryDatabase
from hippynn.experiment.assembly import assemble_for_training
from hippynn.experiment.controllers import RaiseBatchSizeOnPlateau,PatienceController
from hippynn.experiment import setup_training
from hippynn.experiment import train_model
import seqm
from seqm.basics import parameterlist

from numba import cuda
### keeps SCF loops silent ###
seqm.seqm_functions.scf_loop.debug = False

hippynn.interfaces.pyseqm_interface.check.debug = True

### maximum allowed SCF iterations ###
seqm.seqm_functions.scf_loop.MAX_ITER = 95

# torch.cuda.set_device(0) # Don't try this if you want CPU training!

from torch.profiler import profile, record_function, ProfilerActivity

import matplotlib
matplotlib.use("agg")

Autosaving every 50 seconds


/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Decorating your function! <function KSA_XL_BOMD.one_step at 0x7f538ba65550>


In [ ]:


########
current_dir = "/home/maxim/Projects/TESTS/pyseqm_tests/HIPNN_TRAIN/comb_t1x_ani_bse/"
os.chdir(current_dir)

def main():

    ### directory csv with semiempirical parameters ###
    parameter_file_dir = "/home/maxim/Projects/git2/PYSEQM_dev/seqm/params/"
    
    ### directory with training set ###
    #dataset_path = "/home/maxim/Projects/TESTS/pyseqm_tests/HIPNN_TRAIN/training_set/" 
    dataset_path = "/home/maxim/Projects/git2/PYSEQM_dev/examples/hipnn_train_reactive/combined_datasets/" 
    
    ### Prefix for arrays in folder ###
    dataset_name = ''

    ### folder with models and plots ###
    netname = 'TEST1'
    dirname = netname
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    else:
        pass
        #raise ValueError("Directory {} already exists!".format(dirname))
    os.chdir(dirname)

    TAG = 0 #False (0): first run, True(n): continue

    dtype=torch.float64
    torch.set_default_dtype(dtype)
    #device = torch.device('cuda')
    DEVICE = 'cuda'
    #DEVICE = 'cpu'


    ### list of parameters to be learned ###
    #"""
    learned = [
               'U_ss', 'U_pp',
               'zeta_s', 'zeta_p',
               'beta_s', 
               'beta_p',
               'g_ss',
               'g_sp', 'g_pp', 'g_p2',
               'h_sp',
               'alpha',
            'Gaussian1_K', #'Gaussian2_K', #'Gaussian3_K','Gaussian4_K',
            'Gaussian1_L', #'Gaussian2_L', #'Gaussian3_L','Gaussian4_L',
            'Gaussian1_M', #'Gaussian2_M', #'Gaussian3_M','Gaussian4_M',
          ]
    #"""

    
    ### SEQM parameters ###
    seqm_parameters = {
                "method": "PM6_SP",  # AM1, MNDO, PM3#
                "scf_eps": 20.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                #"scf_converger": [0, 0.4],
                "scf_converger": [1, 0.7, 0.95, 30], # converger used for scf loop
                                           # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                           # [1], adaptive mixing
                                           # [2], adaptive mixing, then pulay
                "sp2": [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,[True, eps] or [False], eps for SP2 conve criteria
                "elements": [0, 1, 6, 7, 8],
                "learned": learned,  # parameterlist[method], #['U_ss'], # learned parameters name list, e.g ['U_ss']
                "parameter_file_dir": parameter_file_dir + "/",  # file directory for other required parameters
                "pair_outer_cutoff": 1.0e10,  # consistent with the unit on coordinates
                "scf_backward": 2, # 0: Hellmann–Feynman theorem, 1: recursive formula, 2: backpropagate through SCF (needed for training)
                "Hf_flag": False,
                'UHF' : True, # use unrestricted HF
                'BS' : True, 
                'HIPNN_automatic_doublet': True, # assign doublet state to molecules with odd number of electrons
                }

    # Log the output of python to `training_log.txt`
    with hippynn.tools.log_terminal("training_log_tag_%d.txt" % TAG,'wt'):# and torch.autograd.set_detect_anomaly(True):

        ### Hyperparameters for the network ###
        network_params = {
            "possible_species": [0,1,6,7,8],   # Z values of the elements
            'n_features': 64,                     # Number of neurons at each layer
            "n_sensitivities": 18,                # Number of sensitivity functions in an interaction layer
            "dist_soft_min": 0.65,  # qm7 1.7  qm9 .85  AL100 .85
            "dist_soft_max": 2.8,  # qm7 10.  qm9 5.   AL100 5.
            "dist_hard_max": 3.6,  # qm7 15.  qm9 7.5  AL100 7.5
            "n_interaction_layers": 2,            # Number of interaction blocks #2
            "n_atom_layers": 3,                   # Number of atom layers in an interaction block #3
        }


        ### Define a model ###

        species = inputs.SpeciesNode(db_name="Z")

        positions = inputs.PositionsNode(db_name="R")

        network = networks.Hipnn("HIPNN_seqm", (species, positions), module_kwargs = network_params)

        n_target_peratom = len(seqm_parameters["learned"])

        decay_factor = 1e-4
        par_atom = HChargeNode("SEQM_Atom_Params",network,module_kwargs=dict(n_target=n_target_peratom,first_is_interacting=True))
        with torch.no_grad():
            for layer in par_atom.torch_module.layers:
                layer.weight.data *= decay_factor
                layer.bias.data *= decay_factor

        seqm_par = par_atom.atom_charges

        lenergy = SEQM_AllNode("SEQM_Energy",(par_atom, positions, species),seqm_parameters, decay_factor = 1.0e-4)

        molecule_energy = lenergy.mol_energy
        #molecule_energy = lenergy.Etot_m_Eiso

        gradient  = physics.GradientNode("gradients", (molecule_energy, positions), sign=-1)

        notconverged = lenergy.notconverged
        scale = ScaleNode("Scale", (notconverged,))

        # gradient.db_name='Gradient_ev'
        # molecule_energy.db_name="EtEi"
    
        gradient.db_name='F_DFT'
        #molecule_energy.db_name="EATOMIZ_DFT_no_outlier"
        molecule_energy.db_name="Et_mNONd3h4bias"
        
        


        mol_mask = SEQM_MolMaskNode("SEQM_MolMask", notconverged)
        atom_mask = AtomMaskNode("Atom_Mask", species)
        gradient_pred = SEQM_MaskOnMolAtomNode("SEQM_MaskMolAtom_Pred", (gradient, mol_mask, atom_mask)).pred
        gradient_true = SEQM_MaskOnMolAtomNode("SEQM_MaskMolAtom_True", (gradient.true, mol_mask.pred, atom_mask.pred))

        molecule_energy_pred = SEQM_MaskOnMolNode("SEQM_MaskMol_Pred", (molecule_energy, mol_mask)).pred
        molecule_energy_true = SEQM_MaskOnMolNode("SEQM_MaskMol_True", (molecule_energy.true, mol_mask.pred))


        ### define loss quantities ###

        rmse_gradient = loss.MSELoss(gradient_pred, gradient_true) ** (1./2.)
        rmse_mol_energy = loss.MSELoss(molecule_energy_pred, molecule_energy_true) ** (1. / 2.)

        #rmse_energy_grad = rmse_gradient + rmse_mol_energy

        mae_gradient = loss.MAELoss(gradient_pred, gradient_true)
        mae_mol_energy = loss.MAELoss(molecule_energy_pred, molecule_energy_true)

        #mae_energy_grad = mae_gradient + mae_mol_energy
        
        rsq_gradient = loss.Rsq(gradient_pred, gradient_true)
        rsq_mol_energy = loss.Rsq(molecule_energy_pred, molecule_energy_true)

        ### SLIGHTLY MORE ADVANCED USAGE

        #pred_per_atom = physics.PerAtom("PeratomPredicted",(molecule_energy,species)).pred
        #true_per_atom = physics.PerAtom("PeratomTrue",(molecule_energy.true,species.true))

        pred_per_atom1 = physics.PerAtom("PeratomPredicted",(molecule_energy,species))
        true_per_atom1 = physics.PerAtom("PeratomTrue",(molecule_energy.true,species.true))
        pred_per_atom = SEQM_MaskOnMolNode("SEQM_PerAtom_Pred", (pred_per_atom1, mol_mask)).pred
        true_per_atom = SEQM_MaskOnMolNode("SEQM_PerAtom_True", (true_per_atom1.pred, mol_mask.pred))
        mae_per_atom = loss.MAELoss(pred_per_atom,true_per_atom)
        rmse_per_atom = loss.MSELoss(pred_per_atom,true_per_atom) ** (1. / 2.)

        rmse_par = loss.MeanSq(seqm_par.pred)

        ### END SLIGHTLY MORE ADVANCED USAGE

        #loss_error = (rmse_energy_grad + mae_energy_grad) + rmse_par*8.0
        #loss_error = (0.3*(rmse_gradient + mae_gradient) + (rmse_mol_energy + mae_mol_energy)) + rmse_par*5.0
        #loss_error = (0.5*(rmse_gradient + mae_gradient) + rmse_mol_energy + mae_mol_energy) + rmse_par*10.0
        #loss_error = ((mae_gradient + 0.3*rmse_gradient)) #+ rmse_par*1.0
        loss_error = rmse_gradient + mae_gradient + 0.2*(rmse_mol_energy + mae_mol_energy) + 10.0*rmse_par #+ 0.2*(mae_mol_energy) #+ 0.5*rmse_par

        #rbar = loss.Mean.of_node(hierarchicality)
        l2_reg = loss.l2reg(network)
        
        loss_regularization = 1.0e-6 * loss.Mean(l2_reg) #+ rbar    # L2 regularization and hierarchicality regularization

        train_loss = loss_error*scale.pred + loss_regularization

        # Validation losses are what we check on the data between epochs -- we can only train to
        # a single loss, but we can check other metrics too to better understand how the model is training.
        # There will also be plots of these things over time when training completes.
        validation_losses = {
            "TperAtom RMSE":rmse_per_atom,
            "TperAtom MAE" : mae_per_atom,
            "Force-RMSE"   : rmse_gradient,
            "Force-MAE"    : mae_gradient,
            "Force-RSQ"    : rsq_gradient,
            "MolEn-RMSE"   : rmse_mol_energy,
            "MolEn-MAE"    : mae_mol_energy,
            "MolEn-RSQ"    : rsq_mol_energy,
            "L2Reg"        : l2_reg,
            "Loss-Err"     : loss_error,
            "Loss-Reg"     : loss_regularization,
            "Loss"         : train_loss,
        }
        early_stopping_key = "Loss-Err"



        plot_maker = plotting.PlotMaker(
            # Simple plots which compare the network to the database

            #plotting.Hist2D.compare(molecule_energy, saved=True),
            plotting.Hist2D(molecule_energy_true, molecule_energy_pred,
                            xlabel="True EtEi",ylabel="Predicted EtEi",
                            saved="EtEi.png"),
            plotting.Hist2D(gradient_true, gradient_pred,
                            xlabel="True Force",ylabel="Predicted Force",
                            saved="grad.png"),

            #Slightly more advanced control of plotting!
            plotting.Hist2D(true_per_atom,pred_per_atom,
                            xlabel="True Energy/Atom",ylabel="Predicted Energy/Atom",
                            saved="PerAtomEn.png"),

            #plotting.HierarchicalityPlot(hierarchicality.pred,
            #                             molecule_energy.pred - molecule_energy.true,
            #                             saved="HierPlot.pdf"),
            plot_every=1,   # How often to make plots -- here, epoch 0, 10, 20...
        )

        if TAG==0: #TRAINING FROM SCRATCH


            training_modules, db_info = \
                assemble_for_training(train_loss,validation_losses,plot_maker=plot_maker)
            training_modules[0].print_structure()

    # ----------------- #
    # Step 3: RUN MODEL #
    # ----------------- #

            database_params = {
                'name': dataset_name,                            # Prefix for arrays in folder
                'directory': dataset_path,
                'quiet': False,                           # Quiet==True: suppress info about loading database
                'seed': 1,                       # Random seed for data splitting
                #'test_size': 0.1,                # Fraction of data used for testing
                #'valid_size':0.1,
                **db_info                 # Adds the inputs and targets names from the model as things to load
            }


            database = DirectoryDatabase(**database_params)
            
            ### a fraction of the data set to ignore (i.e., 0.9 means to ignore 90% of the data set and use 10% for train/test/validation) ###
            database.make_random_split("ignore",0.7)
            del database.splits['ignore']
            database.make_trainvalidtest_split(test_size=0.1,valid_size=0.1)

            #from hippynn.pretraining import set_e0_values
            #set_e0_values(henergy,database,energy_name="T_transpose",trainable_after=False)

            init_lr = 1.0e-4
            optimizer = torch.optim.Adam(training_modules.model.parameters(),lr=init_lr)



            scheduler =  RaiseBatchSizeOnPlateau(optimizer=optimizer,
                                                max_batch_size=60,
                                                patience=4,
                                                factor=0.5
                                                )

            controller = PatienceController(optimizer=optimizer,
                                            scheduler=scheduler,
                                            batch_size=60,
                                            eval_batch_size=60,
                                            max_epochs=200,
                                            termination_patience=20,
                                            fraction_train_eval=0.1,
                                            stopping_key=early_stopping_key,
                                            )

            scheduler.set_controller(controller)

            experiment_params = hippynn.experiment.SetupParams(
                controller = controller,
                device=DEVICE,
            )
            print(experiment_params)

            # Parameters describing the training procedure.

            training_modules, controller, metric_tracker  = setup_training(training_modules=training_modules,
                                                            setup_params=experiment_params)
            
        if TAG>0: #CONTINUE INTERRUPTED TRAINING
            from hippynn.experiment.serialization import load_checkpoint_from_cwd, load_checkpoint
            from hippynn.experiment import train_model
            #load best model
            #structure = load_checkpoint_from_cwd()
            
            #load last model
            structure = load_checkpoint("experiment_structure.pt", "best_checkpoint.pt")
            training_modules = structure["training_modules"]
            training_modules[0].print_structure()
            database = structure["database"]
            
            ### a fraction of the data set to ignore (i.e., 0.9 means to ignore 90% of the data set and use 10% for train/test/validation) ###
            database.make_random_split("ignore",0.5)
            del database.splits['ignore']
            database.make_trainvalidtest_split(test_size=0.1,valid_size=0.1)
            #controller = structure["controller"]
            
            init_lr = 2.0e-5
            optimizer = torch.optim.Adam(training_modules.model.parameters(),lr=init_lr)
            
            scheduler =  RaiseBatchSizeOnPlateau(optimizer=optimizer,
                                                max_batch_size=80,
                                                patience=3,
                                                factor=0.5)
            
            controller = PatienceController(optimizer=optimizer,
                                            scheduler=scheduler,
                                            batch_size=80,
                                            eval_batch_size=70,
                                            max_epochs=200,
                                            termination_patience=20,
                                            fraction_train_eval=0.1,
                                            stopping_key=early_stopping_key,
                                            )
            
            metric_tracker = structure["metric_tracker"]
    
    from hippynn.experiment import train_model
    
    store_all_better=True
    store_best=True
    if isinstance(training_modules[0], torch.nn.DataParallel):
        seqm_module = training_modules[0].module.node_from_name('SEQM_Energy').torch_module
    else:
        seqm_module = training_modules[0].node_from_name('SEQM_Energy').torch_module
    callbacks = [update_scf_eps(seqm_module, 0.9),
                    save_and_stop_after(training_modules, controller, metric_tracker, store_all_better, store_best, [2,0,0,0])]
    
    train_model(training_modules=training_modules,
                database=database,
                controller=controller,
                metric_tracker=metric_tracker,
                callbacks=callbacks,batch_callbacks=None,
                store_all_better=store_all_better,
                store_best=store_best)

if __name__=="__main__":
    main()



Determined Inputs: ["Species(db_name='Z')", "Species(db_name='Z')-true", 'SEQM_Energy.mol_energy-true', "Positions(db_name='R')"]
Determined Outputs: ['L^P_Reg(HIPNN_seqm,p=2)', 'SEQM_Atom_Params.atom_charges', 'SEQM_MaskMolAtom_Pred', 'PeratomTrue', 'Atom_Mask', 'Scale', 'SEQM_MolMask', 'SEQM_MaskMol_Pred', 'SEQM_PerAtom_Pred']
Determined Targets: ['gradients', 'SEQM_Energy.mol_energy']
Device was not specified. Attempting to default to device: cuda:0
Inputs:
	 I0 : SpeciesNode('Species(db_name='Z')')<0x7f538b98a610>
	 I1 : LossTrueNode('Species(db_name='Z')-true')<0x7f538b9c90a0>
	 I2 : LossTrueNode('SEQM_Energy.mol_energy-true')<0x7f538b9bc6d0>
	 I3 : PositionsNode('Positions(db_name='R')')<0x7f538b98a5e0>
Outputs:
	 O0 : _LPReg('L^P_Reg(HIPNN_seqm,p=2)')<0x7f538b9d2a00>
	 O1 : IndexNode('SEQM_Atom_Params.atom_charges')<0x7f538b9a1a00>
	 O2 : SEQM_MaskOnMolAtomNode('SEQM_MaskMolAtom_Pred')<0x7f538b9bca30>
	 O3 : PerAtom('PeratomTrue')<0x7f538b9c9070>
	 O4 : AtomMaskNode('Atom_Mask')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Data types:
{'R': dtype('float64'), 'Z': dtype('int64'), 'F_DFT': dtype('float64'), 'Et_mNONd3h4bias': dtype('float64')}
All arrays:
--------------------------------------------------------------------------------------
| Name               | dtype              | shape                                    |
--------------------------------------------------------------------------------------
| R                  | dtype('float64')   | (149415, 20, 3)                          |
| Z                  | dtype('int64')     | (149415, 20)                             |
| F_DFT              | dtype('float64')   | (149415, 20, 3)                          |
| Et_mNONd3h4bias    | dtype('float64')   | (149415,)                                |
--------------------------------------------------------------------------------------
Database: Using auto-generated data indices
Arrays for split: ignore
--------------------------------------------------------------------------------------
| Name         

Training Batches:   0%|          | 0/598 [00:00<?, ?batch/s]/home/maxim/Projects/git2/PYSEQM_dev/seqm/seqm_functions/make_dm_guess.py:291: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/TensorShape.cpp:3277.)
  v_lumo = v[:,0].gather(2, molecule.nocc[:,0].unsqueeze(0).unsqueeze(0).T.repeat(1,v.shape[-1],1))


did not converge 2


/home/maxim/Projects/git2/PYSEQM_dev/seqm/seqm_functions/scf_loop.py:1386: UserWarning: SCF for 2/60 molecules doesn't converge after 95 iterations
  warnings.warn("SCF for %d/%d molecules doesn't converge after %d iterations" % (nnot, nmol, MAX_ITER))
Training Batches:   1%|          | 6/598 [03:10<5:08:57, 31.31s/batch]

did not converge 1


/home/maxim/Projects/git2/PYSEQM_dev/seqm/seqm_functions/scf_loop.py:1386: UserWarning: SCF for 1/60 molecules doesn't converge after 95 iterations
  warnings.warn("SCF for %d/%d molecules doesn't converge after %d iterations" % (nnot, nmol, MAX_ITER))
Training Batches:   2%|▏         | 10/598 [05:17<5:05:31, 31.18s/batch]

did not converge 1


Training Batches:   2%|▏         | 12/598 [06:18<5:01:21, 30.86s/batch]